In [1]:
import pandas as pd
import numpy as np
pd.set_option('display.max_columns', 999)
pd.set_option('display.max_rows', 22)

df_fanfooty = pd.read_csv('inputs/match_data_20200302-174312.csv')
df_fanfooty = df_fanfooty[df_fanfooty['SC'] != '-']
df_fanfooty['SC'] = df_fanfooty['SC'].astype('int')
# Identify when a player is injured
injured_tags = [
    'sore',
    'injured',
    'longterminjured',
    'concussed',
    'heart'
]

def get_injured_status(row):
    if (row['Tag'] in injured_tags or row['Tag 2'] in injured_tags) and row['SC'] < 80:
        return True
    else:
        return False

df_fanfooty['Injured'] = df_fanfooty.apply(lambda row: get_injured_status(row), axis=1)


C:\ProgramData\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3058: DtypeWarning: Columns (9,31) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [2]:
current_season = 2020

df_previous_2_seasons = df_fanfooty[(df_fanfooty['Year'].isin([current_season - 1, current_season - 2])) & (df_fanfooty['Injured'] == False)]
df_previous_2_seasons

,Fanfooty Match ID,Round,Year,Player ID,First Name,Surname,Team,null,DT,SC,null2,null3,null4,Kicks,Handballs,Marks,Tackles,Hitouts,Frees for,Frees against,Goals,Behinds,Not sure,Tag,Tag Notes,Tag 2,Tag 2 Notes,null5,null6,null7,null8,Position,Jumper Number,null9,null10,null11,DT own %,SC own %,AF own %,null12,AF Breakeven,null13,Contested Possessions,Clearances,Clangers,Disposal efficiency,Time on ground,Injured
70362,http://live.fanfooty.com.au/game/matchcentre.h...,R1,2018,290847.0,Dustin,Martin,RI,21,110,139,137,90,123,20,12,5,1,0,1,1,1,3,Full Time,star,%P and %M plus %s,shovel,Starting in central midfield,800000.0,NaN,800000.0,NaN,Midfielder,4.0,0.0,0.0,0.0,6.62,6.62,6.62,800000.0,NaN,NaN,17.0,6.0,3.0,71.0,87.0,False
70363,http://live.fanfooty.com.au/game/matchcentre.h...,R1,2018,250395.0,Jack,Riewoldt,RI,25,100,101,121,86,104,12,6,7,1,0,1,0,4,2,Full Time,x-factor,%s from %D and %M,spearhead,Starting at FF on Weitering,544000.0,NaN,544000.0,NaN,Forward,8.0,0.0,0.0,0.0,0.00,0.00,0.00,544000.0,NaN,NaN,5.0,0.0,2.0,66.0,96.0,False
70364,http://live.fanfooty.com.au/game/matchcentre.h...,R1,2018,290623.0,Josh,Caddy,RI,21,99,84,121,82,98,13,6,5,2,0,5,0,3,2,Full Time,spearhead,%D and %M with %T... %s as well... umps paid h...,NaN,NaN,525000.0,NaN,525000.0,NaN,Midfielder,22.0,0.0,0.0,0.0,0.00,0.00,0.00,525000.0,NaN,NaN,11.0,4.0,2.0,73.0,80.0,False
70365,http://live.fanfooty.com.au/game/matchcentre.h...,R1,2018,992374.0,Daniel,Butler,RI,27,88,109,114,66,84,10,3,2,7,0,3,1,3,0,Full Time,pocket,%s from %D and %M with %T... umps paid him %4F...,NaN,NaN,394000.0,NaN,394000.0,NaN,Forward,7.0,0.0,0.0,0.0,0.00,0.00,0.00,394000.0,NaN,NaN,7.0,0.0,3.0,76.0,76.0,False
70366,http://live.fanfooty.com.au/game/matchcentre.h...,R1,2018,990827.0,Jack,Graham,RI,38,87,100,120,54,79,7,6,0,13,0,1,0,0,1,Full Time,wing,%P and %T plus %s... Rotating in midfield,NaN,NaN,450000.0,NaN,450000.0,NaN,Midfielder,34.0,0.0,0.0,0.0,13.24,13.24,13.24,450000.0,NaN,NaN,9.0,2.0,0.0,76.0,69.0,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
88571,http://live.fanfooty.com.au/game/matchcentre.h...,GF,2019,261374.0,Sam J.,Reid,WS,4,47,55,40,36,51,5,7,2,3,0,0,0,0,0,Full Time,utility,%D and %M with %T... Mostly in defence,NaN,NaN,0.0,NaN,0.0,NaN,Forward,50.0,0.0,0.0,0.0,0.00,0.00,0.00,0.0,NaN,NaN,5.0,1.0,2.0,66.0,96.0,False
88572,http://live.fanfooty.com.au/game/matchcentre.h...,GF,2019,280762.0,Shane,Mumford,WS,8,42,62,38,38,48,2,8,0,1,15,1,0,0,0,Full Time,ruck,%O including %B... First ruck,NaN,NaN,0.0,NaN,0.0,NaN,Ruck,41.0,0.0,0.0,0.0,0.00,0.00,0.00,0.0,NaN,NaN,8.0,2.0,2.0,70.0,75.0,False
88573,http://live.fanfooty.com.au/game/matchcentre.h...,GF,2019,290675.0,Daniel,Lloyd,WS,6,38,54,23,34,52,6,9,1,1,0,0,2,0,1,Full Time,wing,%D and %s... Playing a half forward role,NaN,NaN,0.0,NaN,0.0,NaN,Midfielder,38.0,0.0,0.0,0.0,0.00,0.00,0.00,0.0,NaN,NaN,8.0,2.0,4.0,66.0,76.0,False
88574,http://live.fanfooty.com.au/game/matchcentre.h...,GF,2019,271015.0,Matthew,de Boer,WS,3,31,21,16,31,46,6,6,3,0,0,0,3,0,1,Full Time,tagger,%O and %M plus %s... not helped by %F... Doing...,NaN,NaN,0.0,NaN,0.0,NaN,Midfielder,24.0,0.0,0.0,0.0,0.00,0.00,0.00,0.0,NaN,NaN,1.0,0.0,6.0,66.0,79.0,False


In [3]:
df_season_summary = df_previous_2_seasons.pivot_table(index='Player ID', values='SC', columns='Year', aggfunc=[len, np.sum, np.mean])
df_season_summary.columns = df_season_summary.columns.map('{0[1]}_{0[0]}'.format)
df_season_summary = df_season_summary.reset_index()
column_names = df_season_summary.columns.tolist()
new_column_names = []

for column in column_names:
    column = column.replace("len", "games")
    column = column.replace("sum", "total")
    column = column.replace("mean", "average")
    new_column_names.append(column)

df_season_summary.columns = new_column_names
df_season_summary = df_season_summary[['Player ID', '2018_games', '2018_total', '2018_average', '2019_games', '2019_total', '2019_average']]
df_season_summary

,Player ID,2018_games,2018_total,2018_average,2019_games,2019_total,2019_average
0,210012.0,16.0,1449.0,90.562500,17.0,1326.0,78.000000
1,220001.0,19.0,2019.0,106.263158,24.0,2226.0,92.750000
2,220047.0,19.0,1580.0,83.157895,19.0,1401.0,73.736842
3,220096.0,10.0,1002.0,100.200000,6.0,512.0,85.333333
4,220110.0,12.0,1066.0,88.833333,NaN,NaN,NaN
...,...,...,...,...,...,...,...
757,1008855.0,NaN,NaN,NaN,7.0,354.0,50.571429
758,1008882.0,11.0,490.0,44.545455,21.0,1260.0,60.000000
759,1008893.0,1.0,37.0,37.000000,8.0,436.0,54.500000
760,1010841.0,4.0,210.0,52.500000,4.0,217.0,54.250000


In [4]:
df_sc = pd.read_csv('inputs/sc_fanfooty_combined.csv', index_col=0)
df_sc['sc_name'] = df_sc['sc_first_name'] + ' ' + df_sc['sc_last_name']
df_sc = df_sc[['sc_id', 'sc_feed_id', 'sc_name', 'sc_team', 'sc_position']]
df_sc = df_sc.merge(df_season_summary, how='left', left_on='sc_feed_id', right_on='Player ID')
df_sc = df_sc[['sc_id', 'sc_feed_id', 'sc_name', 'sc_team', 'sc_position', '2018_games', '2018_total', '2018_average', '2019_games', '2019_total', '2019_average']]
df_sc

,sc_id,sc_feed_id,sc_name,sc_team,sc_position,2018_games,2018_total,2018_average,2019_games,2019_total,2019_average
0,1,294472,Rory Atkins,ADE,MID,20.0,1532.0,76.600000,20.0,1510.0,75.500000
1,2,293193,Luke Brown,ADE,DEF,18.0,1354.0,75.222222,15.0,1020.0,68.000000
2,3,1004995,Jordon Butts,ADE,DEF,NaN,NaN,NaN,NaN,NaN,NaN
3,4,993796,Ben Crocker,ADE,FWD,10.0,458.0,45.800000,3.0,105.0,35.000000
4,5,294307,Brad Crouch,ADE,MID,NaN,NaN,NaN,21.0,2087.0,99.380952
...,...,...,...,...,...,...,...,...,...,...,...
810,812,996554,Jake Waterman,WCE,FWD,16.0,898.0,56.125000,13.0,738.0,56.769231
811,813,298474,Francis Watson,WCE,DEF,NaN,NaN,NaN,2.0,134.0,67.000000
812,814,1006114,Bailey Williams,WCE,DEF,NaN,NaN,NaN,NaN,NaN,NaN
813,815,292128,Elliot Yeo,WCE,MID,25.0,2678.0,107.120000,23.0,2418.0,105.130435


In [5]:
wanted_rounds = ['R14', 'R15', 'R16', 'R17', 'R18', 'R19', 'R20', 'R21', 'R22', 'R23', 'EF', 'SF', 'PF', 'GF']
df_final_half = df_fanfooty[(df_fanfooty['Year'] == current_season - 1) & (df_fanfooty['Injured'] == False) & (df_fanfooty['Round'].isin(wanted_rounds))]

df_final_half_summary = df_final_half.pivot_table(index='Player ID', values='SC', columns='Year', aggfunc=[len, np.sum, np.mean])
df_final_half_summary.columns = df_final_half_summary.columns.map('{0[1]}_{0[0]}'.format)
df_final_half_summary = df_final_half_summary.reset_index()
column_names = df_final_half_summary.columns.tolist()
new_column_names = []

for column in column_names:
    column = column.replace("len", "games_lhalf")
    column = column.replace("sum", "total_lhalf")
    column = column.replace("mean", "average_lhalf")
    new_column_names.append(column)

df_final_half_summary.columns = new_column_names
# df_final_half_summary = df_final_half_summary[['Player ID', '2018_games', '2018_total', '2018_average', '2019_games', '2019_total', '2019_average', 'games_lhalf', 'total_lhalf', 'average_lhalf']]
df_sc = df_sc.merge(df_final_half_summary, how='left', left_on='sc_feed_id', right_on='Player ID')
df_sc = df_sc[['sc_id', 'sc_feed_id', 'sc_name', 'sc_team', 'sc_position', '2018_games', '2018_total', '2018_average', '2019_games', '2019_total', '2019_average', '2019_games_lhalf', '2019_total_lhalf', '2019_average_lhalf']]
df_sc

,sc_id,sc_feed_id,sc_name,sc_team,sc_position,2018_games,2018_total,2018_average,2019_games,2019_total,2019_average,2019_games_lhalf,2019_total_lhalf,2019_average_lhalf
0,1,294472,Rory Atkins,ADE,MID,20.0,1532.0,76.600000,20.0,1510.0,75.500000,7.0,508.0,72.571429
1,2,293193,Luke Brown,ADE,DEF,18.0,1354.0,75.222222,15.0,1020.0,68.000000,9.0,628.0,69.777778
2,3,1004995,Jordon Butts,ADE,DEF,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,4,993796,Ben Crocker,ADE,FWD,10.0,458.0,45.800000,3.0,105.0,35.000000,3.0,105.0,35.000000
4,5,294307,Brad Crouch,ADE,MID,NaN,NaN,NaN,21.0,2087.0,99.380952,9.0,863.0,95.888889
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
810,812,996554,Jake Waterman,WCE,FWD,16.0,898.0,56.125000,13.0,738.0,56.769231,8.0,475.0,59.375000
811,813,298474,Francis Watson,WCE,DEF,NaN,NaN,NaN,2.0,134.0,67.000000,2.0,134.0,67.000000
812,814,1006114,Bailey Williams,WCE,DEF,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
813,815,292128,Elliot Yeo,WCE,MID,25.0,2678.0,107.120000,23.0,2418.0,105.130435,12.0,1274.0,106.166667


In [8]:
df_sc.to_csv('lel.csv')